# 5. Solving Partial Differential Equations

<div id="toc"></div>

In [6]:
import os, sys

In [7]:
sys.path.append('./py')

## 5.1 Finite Difference Methods

### 5.1.1 Reduction of a PDE to a System of ODEs

### 5.1.2 Construction of a Test Problem with Known Discrete Solution

### 5.1.3 Implementation: Forward Euler Method

In [ ]:
# %load py/test_diffusion_pde_exact_linear.py
"""Verify the implementation of the diffusion equation."""

from ode_system_FE import ode_FE
from numpy import linspace, zeros, abs

def rhs(u, t):
    N = len(u) - 1
    rhs = zeros(N+1)
    rhs[0] = dsdt(t)
    for i in range(1, N):
        rhs[i] = (beta/dx**2)*(u[i+1] - 2*u[i] + u[i-1]) + \
                 f(x[i], t)
    rhs[N] = (beta/dx**2)*(2*u[N-1] + 2*dx*dudx(t) -
                           2*u[N]) + f(x[N], t)
    return rhs

def u_exact(x, t):
    return (3*t + 2)*(x - L)

def dudx(t):
    return (3*t + 2)

def s(t):
    return u_exact(0, t)

def dsdt(t):
    return 3*(-L)

def f(x, t):
    return 3*(x-L)


def verify_sympy_ForwardEuler():
    import sympy as sp
    beta, x, t, dx, dt, L = sp.symbols('beta x t dx dt L')
    u = lambda x, t: (3*t + 2)*(x - L)**2
    f = lambda x, t, beta, L: 3*(x-L)**2 - (3*t + 2)*2*beta
    s = lambda t: (3*t + 2)*L**2
    N = 4
    rhs = [None]*(N+1)
    rhs[0] = sp.diff(s(t), t)
    for i in range(1, N):
        rhs[i] = (beta/dx**2)*(u(x+dx,t) - 2*u(x,t) + u(x-dx,t)) + \
                 f(x, t, beta, L)
    rhs[N] = (beta/dx**2)*(u(x-dx,t) + 2*dx*(3*t+2) -
                           2*u(x,t) + u(x-dx,t)) + f(x, t, beta, L)
    for i in range(len(rhs)):
        rhs[i] = sp.simplify(sp.expand(rhs[i])).subs(x, i*dx)
        print rhs[i]
        lhs = (u(x, t+dt) - u(x,t))/dt  # Forward Euler difference
        lhs = sp.simplify(sp.expand(lhs.subs(x, i*dx)))
        print lhs
        print sp.simplify(lhs - rhs[i])
        print '---'

def test_diffusion_exact_linear():
    global beta, dx, L, x  # needed in rhs
    L = 1.5
    beta = 0.5
    N = 4
    x = linspace(0, L, N+1)
    dx = x[1] - x[0]
    u = zeros(N+1)

    U_0 = zeros(N+1)
    U_0[0] = s(0)
    U_0[1:] = u_exact(x[1:], 0)
    dt = 0.1
    print dt

    u, t = ode_FE(rhs, U_0, dt, T=1.2)

    tol = 1E-12
    for i in range(0, u.shape[0]):
        diff = abs(u_exact(x, t[i]) - u[i,:]).max()
        assert diff < tol, 'diff=%.16g' % diff
        print 'diff=%g at t=%g' % (diff, t[i])


if __name__ == '__main__':
    test_diffusion_exact_linear()
    verify_sympy_ForwardEuler()


### 5.1.4 Application: Heat Conduction in a Rod

In [ ]:
# %load py/rod_FE.py
"""Temperature evolution in a rod, computed by a ForwardEuler method."""

from numpy import linspace, zeros, linspace
import time

def rhs(u, t):
    N = len(u) - 1
    rhs = zeros(N+1)
    rhs[0] = dsdt(t)
    for i in range(1, N):
        rhs[i] = (beta/dx**2)*(u[i+1] - 2*u[i] + u[i-1]) + \
                 g(x[i], t)
    i = N
    rhs[i] = (beta/dx**2)*(2*u[i-1] + 2*dx*dudx(t) -
                           2*u[i]) + g(x[N], t)
    return rhs

def dudx(t):
    return 0

def s(t):
    return 323

def dsdt(t):
    return 0

def g(x, t):
    return 0


L = 0.5
beta = 8.2E-5
N = 40
x = linspace(0, L, N+1)
dx = x[1] - x[0]
u = zeros(N+1)

U_0 = zeros(N+1)
U_0[0] = s(0)
U_0[1:] = 283
dt = dx**2/(2*beta)
print 'stability limit:', dt
#dt = 0.00034375

t0 = time.clock()
from ode_system_FE import ode_FE
u, t = ode_FE(rhs, U_0, dt, T=1*60*60)
t1 = time.clock()
print 'CPU time: %.1fs' % (t1 - t0)

# Make movie
import os
os.system('rm tmp_*.png')
import matplotlib.pyplot as plt
plt.ion()
y = u[0,:]
lines = plt.plot(x, y)
plt.axis([x[0], x[-1], 273, s(0)+10])
plt.xlabel('x')
plt.ylabel('u(x,t)')
counter = 0
# Plot each of the first 100 frames, then increase speed by 10x
change_speed = 100
for i in range(0, u.shape[0]):
    print t[i]
    plot = True if i <= change_speed else i % 10 == 0
    lines[0].set_ydata(u[i,:])
    if i > change_speed:
        plt.legend(['t=%.0f 10x' % t[i]])
    else:
        plt.legend(['t=%.0f' % t[i]])
    plt.draw()
    if plot:
        plt.savefig('tmp_%04d.png' % counter)
        counter += 1
    #time.sleep(0.2)


### 5.1.5 Vectorization

### 5.1.6 Using Odespy to Solve the System of ODEs

### 5.1.7 Implicit Methods

In [ ]:
# %load py/rod_BE.py
"""Temperature evolution in a rod, computed by a BackwardEuler method."""

from numpy import linspace, zeros, linspace

def rhs(u, t):
    N = len(u) - 1
    rhs = zeros(N+1)
    rhs[0] = dsdt(t)
    for i in range(1, N):
        rhs[i] = (beta/dx**2)*(u[i+1] - 2*u[i] + u[i-1]) + \
                 g(x[i], t)
    rhs[N] = (beta/dx**2)*(2*u[i-1] + 2*dx*dudx(t) -
                           2*u[i]) + g(x[N], t)
    return rhs

def K(u, t):
    N = len(u) - 1
    K = zeros((N+1,N+1))
    K[0,0] = 0
    for i in range(1, N):
        K[i,i-1] = beta/dx**2
        K[i,i] = -2*beta/dx**2
        K[i,i+1] = beta/dx**2
    K[N,N-1] = (beta/dx**2)*2
    K[N,N] = (beta/dx**2)*(-2)
    return K

def rhs_vec(u, t):
    N = len(u) - 1
    rhs = zeros(N+1)
    rhs[0] = dsdt(t)
    rhs[1:N] = (beta/dx**2)*(u[2:N+1] - 2*u[1:N] + u[0:N-1]) + \
               g(x[1:N], t)
    i = N
    rhs[i] = (beta/dx**2)*(2*u[i-1] + 2*dx*dudx(t) -
                           2*u[i]) + g(x[N], t)
    return rhs

def K_vec(u, t):
    """Vectorized computation of K."""
    N = len(u) - 1
    K = zeros((N+1,N+1))
    K[0,0] = 0
    K[1:N-1] = beta/dx**2
    K[1:N] = -2*beta/dx**2
    K[2:N+1] = beta/dx**2
    K[N,N-1] = (beta/dx**2)*2
    K[N,N] = (beta/dx**2)*(-2)
    return K

def dudx(t):
    return 0

def s(t):
    return 323

def dsdt(t):
    return 0

def g(x, t):
    return 0

L = 0.5
beta = 8.2E-5
N = 40
x = linspace(0, L, N+1)
dx = x[1] - x[0]
u = zeros(N+1)

U_0 = zeros(N+1)
U_0[0] = s(0)
U_0[1:] = 283
dt = dx**2/(2*beta)
print 'stability limit:', dt
dt = 600  # 10 min

import odespy
solver = odespy.BackwardEuler(rhs, f_is_linear=True, jac=K)
solver = odespy.ThetaRule(rhs, f_is_linear=True, jac=K, theta=0.5)
solver.set_initial_condition(U_0)
T = 1*60*60
N_t = int(round(T/float(dt)))
time_points = linspace(0, T, N_t+1)
u, t = solver.solve(time_points)

# Make movie
import os
os.system('rm tmp_*.png')
import matplotlib.pyplot as plt
import time
plt.ion()
y = u[0,:]
lines = plt.plot(x, y)
plt.axis([x[0], x[-1], 273, s(0)+10])
plt.xlabel('x')
plt.ylabel('u(x,t)')
counter = 0
for i in range(0, u.shape[0]):
    print t[i]
    lines[0].set_ydata(u[i,:])
    plt.legend(['t=%.0f' % t[i]])
    plt.draw()
    plt.savefig('tmp_%04d.png' % counter)
    counter += 1
    time.sleep(0.2)


## 5.2 Exercises

### Exercise 5.1: Simulate a diffusion equation by hand  

In [ ]:
"""Hand calc: Verify implementation of the diffusion equation"""

from ode_system_FE import ode_FE
from numpy import linspace, zeros, abs
#from test_diffusion_pde_exact_linear import rhs, u_exact, dudx, s, dsdt, f

def rhs(u, t):
    N = len(u) - 1
    rhs = zeros(N+1)
    rhs[0] = dsdt(t)
    for i in range(1, N):
        rhs[i] = (beta/dx**2)*(u[i+1] - 2*u[i] + u[i-1]) + \
                 g(x[i], t)
    rhs[N] = (beta/dx**2)*(2*u[N-1] + 2*dx*dudx(t) -
                           2*u[N]) + g(x[N], t)
    return rhs

def u_exact(x, t):
    return (3*t + 2)*(x - L)

def dudx(t):
    return (3*t + 2)

def s(t):
    return u_exact(0, t)

def dsdt(t):
    return 3*(-L)

def g(x, t):
    return 3*(x-L)

def test_rod_diffusion_hand():
    global beta, dx, L, x      # needed in rhs
    L = 1.5
    beta = 0.5
    N = 2
    x = linspace(0, L, N+1)
    dx = x[1] - x[0]
    u = zeros(N+1)

    U_0 = zeros(N+1)
    U_0[0] = s(0)
    U_0[1:] = u_exact(x[1:], 0)

    u_hand = zeros((3, len(U_0)))
    u_hand[0,:] = -3.0, -1.5, 0.0    # spatial indices: 0, 1 and 2
    u_hand[1,:] = -3.45, -1.725, 0.0
    u_hand[2,:] = -3.90, -1.95, 0.0

    dt = 0.1

    u, t = ode_FE(rhs, U_0, dt, T=1.2)

    tol = 1E-12
    for i in [0, 1, 2]:
        print u_hand[i,:]
        print u[i,:]
        diff = abs(u_hand[i,:] - u[i,:]).max()
        assert diff < tol, 'diff=%.16g' % diff
        print 'diff=%g at t=%g' % (diff, t[i])

if __name__ == '__main__':
    test_rod_diffusion_hand()

### Exercise 5.2: Compute temperature variations in the round  

In [ ]:
"""Temperature vertically down in the ground (with Forward Euler)."""

from numpy import linspace, zeros, linspace, exp, sin, cos, pi, sqrt
import time

def U(x, t):
    """ True temperature """
    return A + B*exp(-r*x)*sin(omega*t - r*x)

def rhs(u, t):
    N = len(u) - 1
    rhs = zeros(N+1)
    rhs[0] = dsdt(t)
    for i in range(1, N):
        rhs[i] = (beta/dx**2)*(u[i+1] - 2*u[i] + u[i-1]) + \
                 f(x[i], t)
    i = N
    rhs[i] = (beta/dx**2)*(2*u[i-1] - 2*u[i]) + f(x[N], t)
    return rhs

def dudx(t):
    return 0

def s(t):
    return T0 + Ta*sin((2*pi/P)*t)

def dsdt(t):
    return (Ta*2*pi/P)*cos((2*pi/P)*t)

def f(x, t):
    return 0

def I(x):
    """Initial temp distribution"""
    return A + B*exp(-r*x)*sin(-r*x)

def I(x):
    """Initial temp distribution."""
    return A

beta = 1E-6
T0 = 283        # just some choice
Ta = 20         # amplitude of temp osc
P = 24*60*60    # period, 24 hours
A = T0
B = Ta
omega = 2*pi/P
r = sqrt(omega/(2*beta))
L = 2           # depth vertically down in the ground
N = 100
x = linspace(0, L, N+1)
dx = x[1] - x[0]
u = zeros(N+1)

U_0 = zeros(N+1)
U_0[0] = s(0)
U_0[1:] = I(x[1:])

dt = dx**2/float(2*beta)
print 'stability limit:', dt

t0 = time.clock()
from ode_system_FE import ode_FE
T=(24*60*60)*6                  # simulate 6 days
u, t = ode_FE(rhs, U_0, dt, T)
t1 = time.clock()
print 'CPU time: %.1fs' % (t1 - t0)


# Make movie
import os
os.system('rm tmp_*.png')
import matplotlib.pyplot as plt
plt.ion()
lines = plt.plot(x, u[0,:], x, U(x, 0))
plt.axis([x[0], x[-1], 283-30, 283+30])
plt.xlabel('x')
plt.ylabel('u(x,t) and U(x,t)')
counter = 0
for i in range(0, u.shape[0]):
    print t[i]
    lines[0].set_ydata(u[i,:])
    lines[1].set_ydata(U(x, t[i]))
    plt.legend(['t=%.3f' % t[i]])
    plt.draw()
    if i % 10 == 0: # plot every x steps
        #plt.savefig('tmp_%04d.png' % counter)
        counter += 1
    #time.sleep(0.2)

### Exercise 5.3: Compare implicit methods  

In [3]:
"""
Temperature evolution in a rod,
computed by a BackwardEuler method and a 2-step
backward method.
"""

from numpy import linspace, zeros, linspace

def rhs(u, t):
    N = len(u) - 1
    rhs = zeros(N+1)
    rhs[0] = dsdt(t)
    for i in range(1, N):
        rhs[i] = (beta/dx**2)*(u[i+1] - 2*u[i] + u[i-1]) + \
                 f(x[i], t)
    rhs[N] = (beta/dx**2)*(2*u[N-1] + 2*dx*dudx(t) -
                           2*u[N]) + g(x[N], t)
    return rhs

def K(u, t):
    N = len(u) - 1
    K = zeros((N+1,N+1))
    K[0,0] = 0
    for i in range(1, N):
        K[i,i-1] = beta/dx**2
        K[i,i] = -2*beta/dx**2
        K[i,i+1] = beta/dx**2
    K[N,N-1] = (beta/dx**2)*2
    K[N,N] = (beta/dx**2)*(-2)
    return K

def dudx(t):
    return 0

def s(t):
    return 323

def dsdt(t):
    return 0

def g(x, t):
    return 0


L = 0.5
beta = 8.2E-5
N = 40
x = linspace(0, L, N+1)
dx = x[1] - x[0]
u = zeros(N+1)

U_0 = zeros(N+1)
U_0[0] = s(0)
U_0[1:] = 283
dt = dx**2/(2*beta)
print 'stability limit:', dt
dt = 12

import odespy
solver0 = odespy.BackwardEuler(rhs, f_is_linear=True, jac=K)
solver1 = odespy.Backward2Step(rhs, f_is_linear=True, jac=K)
solver2 = odespy.Backward2Step(rhs, f_is_linear=True, jac=K)
solver0.set_initial_condition(U_0)
solver1.set_initial_condition(U_0)
solver2.set_initial_condition(U_0)
T = 1*60*60
N_t = int(round(T/float(dt)))
time_points = linspace(0, T, N_t+1)
u0, t0 = solver0.solve(time_points)
u1, t1 = solver1.solve(time_points)
# Solve with larger time step in the 2-step method
factor = 10
dt = factor*dt
N_t = int(round(T/float(dt)))
time_points = linspace(0, T, N_t+1)
u2, t2 = solver2.solve(time_points)

# Make movie
import os
os.system('rm tmp_*.png')
import matplotlib.pyplot as plt
import time
plt.ion()
lines = plt.plot(x, u0[0,:], x, u1[0,:], x, u2[0,:])
plt.axis([x[0], x[-1], 273, s(0)+10])
plt.xlabel('x')
plt.ylabel('u(x,t)')
counter = 0
for i in range(len(t2)):
    print t2[i]
    lines[2].set_ydata(u2[i,:])
    lines[0].set_ydata(u0[factor*i,:])
    lines[1].set_ydata(u1[factor*i,:])
    plt.title('t=%.3f' % t2[i])
    plt.legend(['BE', 'B2Step', 'B2Step 2*dt'])
    plt.draw()
    time.sleep(0.5)
    plt.savefig('tmp_%04d.png' % counter)
    counter += 1

stability limit: 0.952743902439
0.0
120.0
240.0
360.0
480.0
600.0
720.0
840.0
960.0
1080.0
1200.0
1320.0
1440.0
1560.0
1680.0
1800.0
1920.0
2040.0
2160.0
2280.0
2400.0
2520.0
2640.0
2760.0
2880.0
3000.0
3120.0
3240.0
3360.0
3480.0
3600.0


### Exercise 5.4: Explore adaptive and implicit methods  

In [ ]:
"""
Temperature vertically down in the ground with Forward Euler,
Backward Euler, backward 2-step and RKFehlberg .
"""

from numpy import linspace, zeros, linspace, sin, cos, pi, \
                  exp, sqrt
import time

def rhs(u, t):
    N = len(u) - 1
    rhs = zeros(N+1)
    rhs[0] = dsdt(t)
    for i in range(1, N):
        rhs[i] = (beta/dx**2)*(u[i+1] - 2*u[i] + u[i-1]) + \
                 g(x[i], t)
    i = N
    rhs[i] = (beta/dx**2)*(2*u[i-1] - 2*u[i]) + g(x[N], t)
    return rhs

def K(u, t):
    N = len(u) - 1
    K = zeros((N+1, N+1))
    K[0, 0] = 0
    for i in range(1, N):
        K[i, i-1] = beta/dx**2
        K[i, i]   = -2*beta/dx**2
        K[i, i+1] = beta/dx**2
    K[N, N-1] = (beta/dx**2)*2
    K[N, N]   = (beta/dx**2)*(-2)
    return K

def dudx(t):
    return 0

def s(t):
    return T0 + Ta*sin((2*pi/P)*t)

def dsdt(t):
    return (Ta*2*pi/P)*cos((2*pi/P)*t)

def g(x, t):
    return 0

def U(x, t):
    """ True temperature."""
    return A + B*exp(-r*x)*sin(omega*t - r*x)

def I(x):
    """Initial temp distribution."""
    return U(x, 0)

T0 = 283        # just some choice
Ta = 20         # amplitude of temp osc
P = 24*60*60    # period, 24 hours
A = T0
B = Ta
omega = 2*pi/P
beta = 1E-6
r = sqrt(omega/(2*beta))
L = 2           # depth vertically down in the ground
N = 100
x = linspace(0, L, N+1)
dx = x[1] - x[0]
u = zeros(N+1)

U_0 = zeros(N+1)
U_0[0] = s(0)
U_0[1:] = I(x[1:])
FE_stability_limit = dx**2/float(2*beta)
dt = 1*FE_stability_limit
print 'dt/(FE stability limit): %.1f' % (dt/FE_stability_limit)
T=(24*60*60)*6   # simulate 6 days

import odespy
solver1 = odespy.BackwardEuler(rhs, f_is_linear=True, jac=K)
solver2 = odespy.Backward2Step(rhs, f_is_linear=True, jac=K)
solver3 = odespy.RKFehlberg(rhs, rtol=0.001, atol=0.0001)
solver1.set_initial_condition(U_0)
solver2.set_initial_condition(U_0)
solver3.set_initial_condition(U_0)
N_t = int(round(T/float(dt)))
time_points = linspace(0, T, N_t+1)
u1, t1 = solver1.solve(time_points)
u2, t2 = solver2.solve(time_points)
u3, t3 = solver3.solve(time_points)
print 'N_t=%d, RKF N_t:%d' % (N_t, len(solver3.t_all))
print 'Average dt for RKF: %.2f' % (T/float(len(solver3.t_all)))

if dt <= FE_stability_limit:
    from ode_system_FE import ode_FE
    u4, t4 = ode_FE(rhs, U_0, dt, T)

# Compute error with each method
E1 = 0; E2 = 0; E3 = 0; E_FE = 0
for i in range(len(x)):
    for n in range(len(time_points)):
        if dt <= FE_stability_limit:
            E_FE += (U(x[i], time_points[n]) - u4[n,i])**2
        E1 += (U(x[i], time_points[n]) - u1[n,i])**2
        E2 += (U(x[i], time_points[n]) - u2[n,i])**2
        E3 += (U(x[i], time_points[n]) - u3[n,i])**2
if dt <= FE_stability_limit:
    E_FE = sqrt(dx*dt*E_FE)
E1 = sqrt(dx*dt*E1)
E2 = sqrt(dx*dt*E2)
E3 = sqrt(dx*dt*E3)

#print E0  # gives 31.86 with dt on stability limit
if dt <= FE_stability_limit:
    ref_error = E_FE
else:
    ref_error = 66.31
print  'ref_error:', ref_error

if E1 > (ref_error*10):
    print 'ref_error: %.2f , dt = %.4f at stability lim' % \
                          (ref_error, FE_stability_limit)
    print 'E1 (with BE) too large: %.2f, dt = %.4f' % (E1, dt)
elif E2 > (ref_error*10):
    print 'ref_error: %.2f , dt = %.4f at stability lim' % \
                          (ref_error, FE_stability_limit)
    print 'E2 (with B2Step) too large: %.2f, dt = %.4f' % (E2, dt)
elif E3 > (ref_error*10):
    print 'ref_error: %.2f , dt = %.4f at stability lim' % \
                          (ref_error, FE_stability_limit)
    print 'E3 (with RKF) too large: %.2f, dt = %.4f' % (E3, dt)

# Make movie
import os
os.system('rm tmp_*.png')
import matplotlib.pyplot as plt
plt.ion()
u0 = U(x, 0)
if dt <= FE_stability_limit:
    lines = plt.plot(x, u0, x, u1[0,:], x, u2[0,:], x, u3[0,:], x, u4[0,:])
else:
    lines = plt.plot(x, u0, x, u1[0,:], x, u2[0,:], x, u3[0,:])
plt.axis([x[0], x[-1], 283-30, 283+30])
plt.xlabel('x')
plt.ylabel('u(x,t)')
counter = 0
for i in range(len(t1)):
    u0 = U(x, t1[i])
    lines[0].set_ydata(u0)
    lines[1].set_ydata(u1[i,:])
    lines[2].set_ydata(u2[i,:])
    lines[3].set_ydata(u3[i,:])
    if dt <= FE_stability_limit:
        lines[4].set_ydata(u4[i,:])
    plt.title('t=%.1f days' % (t1[i]/(60*60*24.)))
    if dt <= FE_stability_limit:
        plt.legend(['exact', 'BE', 'B2Step', 'RKF', 'FE'])
    else:
        plt.legend(['exact', 'BE', 'B2Step', 'RKF'])
    plt.draw()
    #time.sleep(0.1)
    plt.savefig('tmp_%04d.png' % counter)
    counter += 1

### Exercise 5.5: Investigate the theta rule  

In [4]:
"""Temperature evolution in a rod, computed by a BackwardEuler method."""

from numpy import linspace, zeros, linspace

def rhs(u, t):
    N = len(u) - 1
    rhs = zeros(N+1)
    rhs[0] = dsdt(t)
    for i in range(1, N):
        rhs[i] = (beta/dx**2)*(u[i+1] - 2*u[i] + u[i-1]) + \
                 f(x[i], t)
    rhs[N] = (beta/dx**2)*(2*u[i-1] + 2*dx*dudx(t) -
                           2*u[i]) + f(x[N], t)
    return rhs

def K(u, t):
    N = len(u) - 1
    K = zeros((N+1,N+1))
    K[0,0] = 0
    for i in range(1, N):
        K[i,i-1] = beta/dx**2
        K[i,i] = -2*beta/dx**2
        K[i,i+1] = beta/dx**2
    K[N,N-1] = (beta/dx**2)*2
    K[N,N] = (beta/dx**2)*(-2)
    return K

def K_vec(u, t):
    """Vectorized computation of K."""
    N = len(u) - 1
    K = zeros((N+1,N+1))
    K[0,0] = 0
    K[1:N-1] = beta/dx**2
    K[1:N] = -2*beta/dx**2
    K[2:N+1] = beta/dx**2
    K[N,N-1] = (beta/dx**2)*2
    K[N,N] = (beta/dx**2)*(-2)
    return K

def dudx(t):
    return 0

def s(t):
    return 423

def dsdt(t):
    return 0

def f(x, t):
    return 0


L = 1
beta = 1
N = 40
x = linspace(0, L, N+1)
dx = x[1] - x[0]
u = zeros(N+1)

U_0 = zeros(N+1)
U_0[0] = s(0)
U_0[1:] = 283
dt = dx**2/(2*beta)
print 'stability limit:', dt
dt = 0.05
#dt = 0.00034375  # ForwardEuler limit

import odespy
solver = odespy.ThetaRule(rhs, f_is_linear=True, jac=K, theta=0.5)
solver.set_initial_condition(U_0)
T = 1.2
N_t = int(round(T/float(dt)))
time_points = linspace(0, T, N_t+1)
u, t = solver.solve(time_points)

# Make movie
import os
os.system('rm tmp_*.png')
import matplotlib.pyplot as plt
import time
plt.ion()
y = u[0,:]
lines = plt.plot(x, y)
plt.axis([x[0], x[-1], 273, 1.2*s(0)])
plt.xlabel('x')
plt.ylabel('u(x,t)')
counter = 0
for i in range(0, u.shape[0]):
    print t[i]
    lines[0].set_ydata(u[i,:])
    plt.legend(['t=%.3f' % t[i]])
    plt.draw()
    plt.savefig('tmp_%04d.png' % counter)
    counter += 1
    time.sleep(0.2)

stability limit: 0.0003125
0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
1.0
1.05
1.1
1.15
1.2


### Exercise 5.6: Compute the diffusion of a Gaussian peak  

In [ ]:
"""
Initially, u has a Gaussian distribution and we
follow the distribution over x = [-1,1] until u
is approximately zero.
"""

from numpy import linspace, zeros, linspace, exp, sqrt, pi

def rhs(u, t):
    N = len(u) - 1
    rhs = zeros(N+1)
    rhs[0] = (2*beta/dx**2)*(u[1] - u[0]) + g(x[0], t)
    for i in range(1, N):
        rhs[i] = (beta/dx**2)*(u[i+1] - 2*u[i] + u[i-1]) + \
                 g(x[i], t)
    rhs[N] = (2*beta/dx**2)*(u[N-1] - u[N]) + g(x[N], t)
    return rhs

def K(u, t):
    N = len(u) - 1
    K = zeros((N+1,N+1))
    K[0,0] = -2*beta/dx**2;
    K[0,1] =  2*beta/dx**2;
    for i in range(1, N):
        K[i,i-1] = beta/dx**2
        K[i,i] = -2*beta/dx**2
        K[i,i+1] = beta/dx**2
    K[N,N-1] = 2*beta/dx**2
    K[N,N] = -2*beta/dx**2
    return K

def I(x):
    """Initial conditions for 1D diffusion"""
    #return (1.0/(sqrt(2*pi)*sigma)*exp(-(x**2)/(2*sigma**2)))
    # Not clean:
    #return (1.0/(sqrt(2*pi)*sigma)*exp(-(x**2)/(2*sigma**2)))
    return exp(-(x**2)/(2*sigma**2))

def g(x, t):
    return 0

sigma = 0.2
a = -1; b = 2*pi
beta = 1
N = 80
x = linspace(a, b, N+1)
dx = x[1] - x[0]
u = zeros(N+1)
U_0 = I(x)

dt = dx**2/(2*beta)
print 'FE stability limit:', dt
print 'stability limit for FE alternative:', dt
# gives: dt = 0.0003125
#dt = 0.01      # choose much longer time step
#dt = 100.0

import odespy
solver0 = odespy.Backward2Step(rhs, f_is_linear=True, jac=K)
solver0 = odespy.ThetaRule(rhs, f_is_linear=True, jac=K, theta=0)
solver0.set_initial_condition(U_0)
T = 10
N_t = int(round(T/float(dt)))
time_points = linspace(0, T, N_t+1)
u0, t0 = solver0.solve(time_points)

# Make movie
import os
os.system('rm -f tmp_*.png')
import matplotlib.pyplot as plt
import time
plt.ion()
lines = plt.plot(x, u0[0,:])
plt.axis([x[0], x[-1], 0, 1.0/(sqrt(2*pi)*sigma)])
plt.xlabel('x')
plt.ylabel('u(x,t)')
counter = 0
for i in range(len(t0)):
    print t0[i], u0[i,round(N/2)]  # u approx at middle (dep on N)
    lines[0].set_ydata(u0[i,:])
    plt.title('t=%.3f' % t0[i])
    plt.legend([solver0.__class__.__name__])
    plt.draw()
    #time.sleep(0.2)
    #plt.savefig('tmp_%04d.png' % counter)
    #counter += 1
raw_input()

### Exercise 5.7: Vectorize a function for computing the area of apolygon  

In [8]:
"""
Computes the area of a polygon from vertex
coordinates only. Vectorization is exploited.
"""

from polyarea import polyarea
from numpy import sum, asarray

def polyarea_vec(x, y):
    x = asarray(x); y = asarray(y)      # since sum requires arrays
    return 0.5*abs(sum(x[:-1]*y[1:]) - sum(y[:-1]*x[1:]) + \
                    x[len(x)-1]*y[0] - y[len(x)-1]*x[0])

def test_polyarea_vec():
    tol = 1E-14
    # pentagon
    x1 = [0, 2, 2, 1, 0]
    y1 = [0, 0, 2, 3, 2]
    #print abs(polyarea(x1, y1) - polyarea_vec(x1, y1))
    assert abs(polyarea(x1, y1) - polyarea_vec(x1, y1)) < tol
    # quadrilateral
    x2 = [0, 2, 2, 0]
    y2 = [0, 0, 2, 2]
    #print abs(polyarea(x2, y2) - polyarea_vec(x2, y2))
    assert abs(polyarea(x2, y2) - polyarea_vec(x2, y2)) < tol
    # triangle
    x3 = [0, 2, 0]
    y3 = [0, 0, 2]
    #print abs(polyarea(x3, y3) - polyarea_vec(x3, y3))
    assert abs(polyarea(x3, y3) - polyarea_vec(x3, y3)) < tol

if __name__ == '__main__':
    test_polyarea_vec()

ImportError: No module named polyarea

### Exercise 5.8: Explore symmetry  

In [ ]:
"""
Initially, u has a Gaussian distribution symmetric around x = 0.
We utilize symmetry here and follow the distribution over
x = [0,1] until u is approximately zero.
"""

from numpy import linspace, zeros, linspace, exp, sqrt, pi

def rhs(u, t):
    N = len(u) - 1
    rhs = zeros(N+1)
    rhs[0] = (2*beta/dx**2)*(u[1] - u[0]) + f(x[0], t)
    for i in range(1, N):
        rhs[i] = (beta/dx**2)*(u[i+1] - 2*u[i] + u[i-1]) + \
                 f(x[i], t)
    rhs[N] = (2*beta/dx**2)*(u[N-1] - u[N]) + f(x[N], t)
    return rhs

def K(u, t):
    N = len(u) - 1
    K = zeros((N+1,N+1))
    K[0,0] = -2*beta/dx**2;
    K[0,1] =  2*beta/dx**2;
    for i in range(1, N):
        K[i,i-1] = beta/dx**2
        K[i,i] = -2*beta/dx**2
        K[i,i+1] = beta/dx**2
    K[N,N-1] = (beta/dx**2)*2
    K[N,N] = (beta/dx**2)*(-2)
    return K

def I(x):
    """Initial conditions for 1D diffusion"""
    return (1.0/(sqrt(2*pi)*sigma)*exp(-(x**2)/(2*sigma**2)))

def f(x, t):
    return 0

sigma = 0.2
sigma = 0.02
a = 0; b = 1
beta = 1
N = 40
N = 400
x = linspace(a, b, N+1)
dx = x[1] - x[0]
u = zeros(N+1)
U_0 = I(x)

dt = dx**2/(2*beta)
print 'stability limit for alternative FE:', dt
# gives: dt = 0.003125
dt = 0.05       # choose much longer time step

import odespy
solver0 = odespy.Backward2Step(rhs, f_is_linear=True, jac=K)
solver0.set_initial_condition(U_0)
T = 3
N_t = int(round(T/float(dt)))
time_points = linspace(0, T, N_t+1)
u0, t0 = solver0.solve(time_points)

# Make movie
import os
os.system('rm tmp_*.png')
import matplotlib.pyplot as plt
import time
plt.ion()
lines = plt.plot(x, u0[0,:])
plt.axis([x[0], x[-1], 0, 1.0/(sqrt(2*pi)*sigma)])
plt.xlabel('x')
plt.ylabel('u(x,t)')
counter = 0
for i in range(len(t0)):
    print t0[i], u0[i,round(N/2)]  # u approx at middle (dep on N)
    lines[0].set_ydata(u0[i,:])
    plt.title('t=%.3f' % t0[i])
    plt.legend(['B2Step'])
    plt.draw()
    time.sleep(0.2)
    #plt.savefig('tmp_%04d.png' % counter)
    #counter += 1

### Exercise 5.9: Compute solutions as t -> infinit  

In [ ]:
"""Compute final and stationary temperature distribution
in a rod by a BackwardEuler method.
"""

from numpy import linspace, zeros, linspace

def rhs(u, t):
    N = len(u) - 1
    rhs = zeros(N+1)
    rhs[0] = dsdt(t)
    for i in range(1, N):
        rhs[i] = (beta/dx**2)*(u[i+1] - 2*u[i] + u[i-1]) + \
                 f(x[i], t)
    rhs[N] = (beta/dx**2)*(2*u[i-1] + 2*dx*dudx(t) -
                           2*u[i]) + f(x[N], t)
    return rhs

def K(u, t):
    N = len(u) - 1
    K = zeros((N+1,N+1))
    K[0,0] = 0
    for i in range(1, N):
        K[i,i-1] = beta/dx**2
        K[i,i] = -2*beta/dx**2
        K[i,i+1] = beta/dx**2
    K[N,N-1] = (beta/dx**2)*2
    K[N,N] = (beta/dx**2)*(-2)
    return K

def dudx(t):
    return 0

def s(t):
    return 423

def dsdt(t):
    return 0

def f(x, t):
    return 0

L = 1
beta = 1
N = 40
x = linspace(0, L, N+1)
dx = x[1] - x[0]
u = zeros(N+1)

U_0 = zeros(N+1)
U_0[0] = s(0)
U_0[1:] = 300
dt = 60                 # choose one large timestep

import odespy
solver = odespy.BackwardEuler(rhs, f_is_linear=True, jac=K)
solver.set_initial_condition(U_0)
#T = 1.2
T = dt
N_t = int(round(T/float(dt)))
time_points = linspace(0, T, N_t+1)
u, t = solver.solve(time_points)

import matplotlib.pyplot as plt
plt.plot(x,u[N_t,:])
plt.axis([x[0], x[-1], 273, 1.2*s(0)])
plt.show()

### Exercise 5.10: Solve a two-point boundary value problem  

In [ ]:
"""
Solve 2-point BVP.
"""

from numpy import linspace, zeros, linspace

def rhs(u, t):
    N = len(u) - 1
    rhs = zeros(N+1)
    rhs[0] = ds1dt(t)
    for i in range(1, N):
        rhs[i] = (beta/dx**2)*(u[i+1] - 2*u[i] + u[i-1]) + \
                 f(x[i], t)
    rhs[N] = ds2dt(t)
    return rhs

def K(u, t):
    N = len(u) - 1
    K = zeros((N+1,N+1))
    K[0,0] = 0
    for i in range(1, N):
        K[i,i-1] = beta/dx**2
        K[i,i] = -2*beta/dx**2
        K[i,i+1] = beta/dx**2
    K[N,N-1] = 0
    K[N,N] = 0
    return K

def dudx(t):
    return 0

def s1(t):
    return 0

def ds1dt(t):
    return 0

def s2(t):
    return 1

def ds2dt(t):
    return 0

def f(x, t):
    return -2

L = 1
beta = 1
N = 40
x = linspace(0, L, N+1)
dx = x[1] - x[0]
u = zeros(N+1)

U_0 = zeros(N+1)
U_0[0] = s1(0)
U_0[1:-1] = 0.5
U_0[-1] = s2(0)
dt = 10                 # choose one large timestep

import odespy
solver = odespy.BackwardEuler(rhs, f_is_linear=True, jac=K)
solver.set_initial_condition(U_0)
#T = 1.2
T = dt
N_t = int(round(T/float(dt)))
time_points = linspace(0, T, N_t+1)
u, t = solver.solve(time_points)

import matplotlib.pyplot as plt
plt.plot(x, u[N_t,:], x, x**2)
plt.legend(['numerical', 'exact'])
plt.show()